In [1]:
import findspark
import os

findspark.init()

import pandas as pd 
import numpy as np
from sklearn import preprocessing
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, mean
from pyspark.sql.types import *

sc = pyspark.SparkContext()

In [2]:
path_train = '/Users/tiegsti.solomon/Downloads'
path_test = '/Users/tiegsti.solomon/Downloads'

# we are taking the training outputs generated by the spark cluster and using the 
# test data provided by outbrain to validate + compare labels

rdd_train = sc.textFile(path_test+"/content_reco_features_.csv")
rdd_test = sc.textFile(path_train+"/content_reco_features_test_1.csv")

print "No. of Lines Train: ", rdd_train.count()
print "No. of Lines Test: ", rdd_test.count()

print "No. of Partitions: " , rdd_train.getNumPartitions() # 2 partitions

No. of Lines Train:  576853
No. of Lines Test:  105293
No. of Partitions:  2


In [3]:
# remove header train
rdd_train_head = rdd_train.filter(lambda l: "uuid" in l)
rdd_train = rdd_train.subtract(rdd_train_head)

# remove header test
rdd_test_head = rdd_test.filter(lambda l: "uuid" in l)
rdd_test = rdd_test.subtract(rdd_test_head)

In [4]:
rdd_train = rdd_train.map(lambda a: a.split(",")).take(100000)
rdd_test = rdd_test.map(lambda a: a.split(",")).take(100000)

In [5]:
from pyspark.sql.functions import udf, col

sqlContext = SQLContext(sc)
train_df = sqlContext.createDataFrame(rdd_train) # schema err
test_df = sqlContext.createDataFrame(rdd_test)
train_df.show(5)

df_train = train_df.drop('_4','_14','_15','_16')
df_test = test_df.drop('_4','_14','_15','_16')

+--------------+-------+------+--------+---+---+---+-------+---+-----+----+---+---+---+----+---+
|            _1|     _2|    _3|      _4| _5| _6| _7|     _8| _9|  _10| _11|_12|_13|_14| _15|_16|
+--------------+-------+------+--------+---+---+---+-------+---+-----+----+---+---+---+----+---+
|45fd37a633fdfa|1746296|140874| 3644856|  2| US|  1|  47246|  0|14742|2691|  4|  1|  0|   0|  0|
|fa58831f4f9c4e|1729862|162985|53650718|  1| US|  1| 827465|  0|15373|2713|  6|  1|  0|   0|  0|
|331cfea3ca7148|  60164|124343|31221391|  1| US|  1|2124518|  1|15965|1634|  4|  1|  1|0.25|  1|
|807cd39278c64c| 384461|319252|50352702|  2| US|  1| 753836|  1|28971|2635| 10|  1|  1| 0.1|  1|
|4536e73cf75581|  60164|152335|53141017|  1| US|  1|6993618|  0|19363|1684|  6|  1|  0|   0|  0|
+--------------+-------+------+--------+---+---+---+-------+---+-----+----+---+---+---+----+---+
only showing top 5 rows



In [6]:
#train_feat = train_df
# test_feat = test_df.columns

train_dfp = df_train.toPandas()
test_dfp =  df_test.toPandas()

print(len(train_dfp))
print(len(test_dfp))

100000
100000


In [7]:
from sklearn.preprocessing import LabelEncoder

le = preprocessing.LabelEncoder()

le.fit(train_dfp['_6'].values)
train_dfp['_6'] = le.transform(train_dfp['_6'])
le.fit(train_dfp['_1'].values)
train_dfp['_1'] = le.transform(train_dfp['_1'])

le.fit(test_dfp['_6'].values)
test_dfp['_6'] = le.transform(test_dfp['_6'])
le.fit(test_dfp['_1'].values)
test_dfp['_1'] = le.transform(test_dfp['_1'])
train_dfp.head(10)

,_1,_2,_3,_5,_6,_7,_8,_9,_10,_11,_12,_13
0,14332,1746296,140874,2,147,1,47246,0,14742,2691,4,1
1,61828,1729862,162985,1,147,1,827465,0,15373,2713,6,1
2,9300,60164,124343,1,147,1,2124518,1,15965,1634,4,1
3,29716,384461,319252,2,147,1,753836,1,28971,2635,10,1
4,14104,60164,152335,1,147,1,6993618,0,19363,1684,6,1
5,48094,1826941,162984,1,147,1,537833,1,15373,2713,6,1
6,2230,1479631,134758,1,147,1,1031635,0,17307,1955,10,1
7,37304,1790442,60620,2,143,1,225382,0,8101,681,5,1
8,12169,1832504,214724,3,29,1,967281,0,24370,1896,4,1
9,59397,1642428,232313,1,147,1,298975,1,25379,3764,6,1


In [8]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

train_targets = train_dfp['_9']

train_df_ = train_dfp.drop('_9', axis=1)

In [9]:
# Combined Features

# combine ad_id - page view count per unique user
train_df_['_14'] = train_df_.apply(lambda x:'%s%s' % (x['_2'],x['_12']),axis=1)
# doc_id - page view count per uuid
train_df_['_15'] = train_df_.apply(lambda x:'%s%s' % (x['_3'],x['_12']),axis=1)
# platform and traffic source
train_df_['_16'] = train_df_.apply(lambda x:'%s%s' % (x['_5'],x['_7']),axis=1) 
# camapign id advertiser id 
train_df_['_17'] = train_df_.apply(lambda x:'%s%s' % (x['_10'],x['_11']),axis=1)
# geo_location , traffic source
train_df_['_18'] = train_df_.apply(lambda x:'%s%s' % (x['_6'],x['_7']),axis=1) 
# campaign id + page view
train_df_['_19'] = train_df_.apply(lambda x:'%s%s' % (x['_10'],x['_12']),axis=1) 
# geo + pv
train_df_['_20'] = train_df_.apply(lambda x:'%s%s' % (x['_6'],x['_12']),axis=1) 
# platform + pv
train_df_['_21'] = train_df_.apply(lambda x:'%s%s' % (x['_5'],x['_12']),axis=1) 




# test data replicate
test_dfp['_14'] = test_dfp.apply(lambda x:'%s%s' % (x['_2'],x['_12']),axis=1)
test_dfp['_15'] = test_dfp.apply(lambda x:'%s%s' % (x['_3'],x['_12']),axis=1)
test_dfp['_16'] = test_dfp.apply(lambda x:'%s%s' % (x['_5'],x['_7']),axis=1) 
test_dfp['_17'] = test_dfp.apply(lambda x:'%s%s' % (x['_10'],x['_11']),axis=1) 
test_dfp['_18'] = test_dfp.apply(lambda x:'%s%s' % (x['_6'],x['_7']),axis=1)
test_dfp['_19'] = test_dfp.apply(lambda x:'%s%s' % (x['_10'],x['_12']),axis=1)
test_dfp['_20'] = test_dfp.apply(lambda x:'%s%s' % (x['_6'],x['_12']),axis=1)
test_dfp['_21'] = test_dfp.apply(lambda x:'%s%s' % (x['_5'],x['_12']),axis=1) 

In [10]:
from sklearn.linear_model import LogisticRegression

log=LogisticRegression(penalty='l1',C=1.0)
log.fit(train_df_, train_targets)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
test_dfp = test_dfp.convert_objects(convert_numeric=True)
test_dfp = test_dfp.dropna(axis=0)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [12]:
y_pred = log.predict(test_dfp)

predicted_rel_prob = log.predict_proba(test_dfp)

from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, log_loss
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt

np.set_printoptions(threshold=np.nan)

y_pred_mod = []

for i in predicted_rel_prob:
    if i[1] >= 0.3:
        y_pred_mod.append('1')
    else:
        y_pred_mod.append('0')
        
y_pred_mod = np.array(y_pred_mod)

print("Accuracy Score LR: ", accuracy_score(train_targets, y_pred_mod))
print("Cross Entropy LR: ", log_loss(train_targets, predicted_rel_prob))

print("Confusion Matrix LR: ", confusion_matrix(train_targets, y_pred_mod))
print("Classification Report: ", metrics.classification_report(train_targets, y_pred_mod))
mean_squared_err = mean_squared_error(train_targets, y_pred)
print("Mean Squared Error: ", mean_squared_err)
print("Root Mean Squared Error: ", sqrt(mean_squared_err))

('Accuracy Score LR: ', 0.65014000000000005)
('Cross Entropy LR: ', 0.50882068596457908)
('Confusion Matrix LR: ', array([[60024, 20949],
       [14037,  4990]]))
('Classification Report: ', u'             precision    recall  f1-score   support\n\n          0       0.81      0.74      0.77     80973\n          1       0.19      0.26      0.22     19027\n\navg / total       0.69      0.65      0.67    100000\n')
('Mean Squared Error: ', 0.19026999999999999)
('Root Mean Squared Error: ', 0.4361994956439083)


In [13]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1")
clf.fit(train_df_, train_targets)
SGDClassifier(alpha=1.0, average=False, learning_rate='optimal',
              class_weight=None, epsilon=0.1,
              fit_intercept=True, l1_ratio=0.15, eta0=0.0,
              loss='log', max_iter=5, n_iter=None,
              n_jobs=1, penalty='l1', power_t=0.5,
              random_state=0,
              tol=None, verbose=0, warm_start=False)

/Library/Python/2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=1.0, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [14]:
y_pred2 = clf.predict(test_dfp)
y_pred_proba = clf.predict_proba(test_dfp)

#np.set_printoptions(threshold=np.nan)

print("Accuracy Score SGD: ", accuracy_score(train_targets, y_pred2))
print("Cross Entropy SGD: ", log_loss(train_targets, y_pred_proba))
print("Confusion Matrix SGD: ", confusion_matrix(train_targets, y_pred2))
print("Classification Report SGD: ", metrics.classification_report(train_targets, y_pred2))
mean_squared_err = mean_squared_error(train_targets, [float(i) for i in y_pred2])
print("Mean Squared Error: ", mean_squared_err)
print("Root Mean Squared Error: ", sqrt(mean_squared_err))

/Library/Python/2.7/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


('Accuracy Score SGD: ', 0.71079000000000003)
('Cross Entropy SGD: ', 9.9889595211722408)
('Confusion Matrix SGD: ', array([[68025, 12948],
       [15973,  3054]]))
('Classification Report SGD: ', u'             precision    recall  f1-score   support\n\n          0       0.81      0.84      0.82     80973\n          1       0.19      0.16      0.17     19027\n\navg / total       0.69      0.71      0.70    100000\n')
('Mean Squared Error: ', 0.28921000000000002)
('Root Mean Squared Error: ', 0.5377824839096194)


In [15]:
# GBR

from sklearn.ensemble import GradientBoostingRegressor

params = {'n_estimators': 100, 'max_depth': 6,
        'learning_rate': 0.1, 'alpha':0.98}
gbr = GradientBoostingRegressor(**params).fit(train_df_, train_targets)

In [16]:
np.set_printoptions(suppress=True)

y_pred3 = gbr.predict(test_dfp)
#y_pred_proba2 = gbr.predict_proba(test_dfp)
y_pred3_mod = []

for i in y_pred3:
    if i >= 0.3:
        y_pred3_mod.append('1')
    else:
        y_pred3_mod.append('0')
        

In [17]:
print("Accuracy Score GBR: ", accuracy_score(train_targets, y_pred3_mod))
print("Cross Entropy GBR: ", log_loss(train_targets, y_pred3))
print("Confusion Matrix GBR: ", confusion_matrix(train_targets, y_pred3_mod))
print("Classification Report GBR: ", metrics.classification_report(train_targets, y_pred3_mod))
mean_squared_err = mean_squared_error(train_targets, y_pred3)
print("Mean Squared Error: ", mean_squared_err)
print("Root Mean Squared Error: ", sqrt(mean_squared_err))

('Accuracy Score GBR: ', 0.57835999999999999)
('Cross Entropy GBR: ', 0.63912518740306234)
('Confusion Matrix GBR: ', array([[50736, 30237],
       [11927,  7100]]))
('Classification Report GBR: ', u'             precision    recall  f1-score   support\n\n          0       0.81      0.63      0.71     80973\n          1       0.19      0.37      0.25     19027\n\navg / total       0.69      0.58      0.62    100000\n')
('Mean Squared Error: ', 0.18066030936470365)
('Root Mean Squared Error: ', 0.42504153839913533)


In [18]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import itertools

X = train_df_
y_true = train_targets
class_names = [0,1]

test_targets = test_dfp['_9']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    

# Compute confusion matrix
cnf_matrix = confusion_matrix(train_targets, y_pred_mod)    

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[60024 20949]
 [14037  4990]]
Normalized confusion matrix
[[ 0.74128413  0.25871587]
 [ 0.7377411   0.2622589 ]]


/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import spline


# # evenly sampled time at 200ms intervals
lr_acc = [0,0.52,0.63,0.64]
gbr_acc = [0,0.31,0.51,0.59]
sgd_acc = [0,0.61,0.81,0.81]
lr_logloss = [0,0.53,0.51,0.51]
gbr_logloss = [0,0.88,0.59,0.62]
sgd_logloss = [0,6.65,6.61,13.18]
x = np.array([0,10000,50000,100000])

# x_smooth = np.linspace(x.min(),x.max())
# y_smooth_lr_acc = spline(x,lr_acc,x_smooth)
# y_smooth_gbr_acc = spline(x,gbr_acc,x_smooth)
# y_smooth_sgd_acc = spline(x,sgd_acc,x_smooth)

# Accuracy 
plt.xlabel('Training Set (No. of rows)')
plt.ylabel('Accuracy')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

# red dashes, blue squares and green triangles
plt.plot(x, lr_acc, 'rs-', x, gbr_acc, 'b^-', x, sgd_acc, 'go-')
plt.show()

In [ ]:
# Cross Entropy

predicted_rel_prob_samp = predicted_rel_prob[0:100]
y_pred3_samp = y_pred3[0:100]
y_pred2_samp = y_pred_proba[0:100]

y_lr = np.array([i[1] for i in predicted_rel_prob_samp])
y_gbr = np.array(y_pred3_samp)
y_sgd = np.array(y_pred2_samp)

#x_smooth_lr = np.linspace(x_lr.min(),x_lr.max())
#x_smooth_gbr = np.linspace(x_gbr.min(),x_gbr.max())
#x_smooth_sgd = np.linspace(x_sgd.min(),x_sgd.max())

x = np.arange(0.0,1.0,0.01)
# y_smooth_lr_logloss = spline(x,x_lr,x_lr)
# y_smooth_gbr_logloss = spline(x,x_gbr,x_smooth_gbr)
# y_smooth_sgd_logloss = spline(x,x_sgd,x_smooth_sgd)

plt.xlabel('Probability (1)')
plt.ylabel('Cross Entropy')
plt.ylim([-0.5,1.5])

# red dashes, blue squares and green triangles
plt.plot(x, y_lr, 'r-', x, y_gbr, 'b-', x, y_sgd, 'g-')
plt.show()

In [ ]:
def ROC(label,result):
    %matplotlib inline
    from sklearn.utils import shuffle
    from sklearn.metrics import roc_curve, auc, precision_score, roc_auc_score
    from sklearn.preprocessing import label_binarize
    import pylab as pl
    import numpy as py
    Y = np.array(label)
    truth = label_binarize(label, classes=[0,1])
    n_classes = truth.shape[1]
    pred = label_binarize(result, classes=[0,1])
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(truth[:, i], pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    # Plot of a ROC curve for a specific class
    pl.figure()
    pl.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
    pl.plot([0, 1], [0, 1], 'k--')
    pl.xlim([0.0, 1.0])
    pl.ylim([0.0, 1.05])
    pl.xlabel('FP')
    pl.ylabel('TP')
    pl.legend(loc="lower right")
    pl.show()

    for i in range(n_classes):
        pl.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                       ''.format(i, roc_auc[i]))

    pl.plot([0, 1], [0, 1], 'k--')
    pl.xlim([0.0, 1.0])
    pl.ylim([0.0, 1.05])
    pl.xlabel('FP')
    pl.ylabel('TP')
    pl.legend(loc="lower right")
    pl.show()


ROC(train_targets[:], y_pred2[:])